# Kompresja bezstratna - Kodowanie Huffmana
*Ivan Kaliadzich 153936*


## Dodawanie bibliotek


In [1]:
import numpy as np
import bitarray
import time
import heapq

## Alphabet i odczytanie pliku

In [2]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

small_text = "to be or not to be"

text_file = open("norm_wiki_sample.txt", "r")
data = text_file.read()
text_file.close()


## Sprawdzenie działania dla małego tekstu

In [3]:
letter_count = {}

for char in small_text:
    if char in letter_count:
        letter_count[char] += 1
    else:
        letter_count[char] = 1

while len(letter_count)>1:
    sorted_items = sorted(letter_count.items(), key=lambda item: (item[1], item[0])) 
    
    smallest_1 = sorted_items[0]
    smallest_2 = sorted_items[1]
    del letter_count[smallest_1[0]]
    del letter_count[smallest_2[0]]
    
    new_key = f'{{{smallest_1[0]},{smallest_2[0]}}}'
    new_value = smallest_1[1] + smallest_2[1]
    
    letter_count[new_key] = new_value
    
    print(sorted_items) 

final_combined_key = list(letter_count.keys())[0]
print(f"Final combined key: {final_combined_key}")

[('n', 1), ('r', 1), ('b', 2), ('e', 2), ('t', 3), ('o', 4), (' ', 5)]
[('b', 2), ('e', 2), ('{n,r}', 2), ('t', 3), ('o', 4), (' ', 5)]
[('{n,r}', 2), ('t', 3), ('o', 4), ('{b,e}', 4), (' ', 5)]
[('o', 4), ('{b,e}', 4), (' ', 5), ('{{n,r},t}', 5)]
[(' ', 5), ('{{n,r},t}', 5), ('{o,{b,e}}', 8)]
[('{o,{b,e}}', 8), ('{ ,{{n,r},t}}', 10)]
Final combined key: {{o,{b,e}},{ ,{{n,r},t}}}


## Sprawdzenie dla "norm_wiki_sample.txt"

In [10]:
class BinaryTree:

    def __init__(self, left_node, right_node):
        self.left_node = left_node
        self.right_node = right_node

    def get_children(self):
        return self.left_node, self.right_node


def calculate_frequency(text, alphabet):
    frequency_dict = dict.fromkeys(alphabet, 0)
    for character in text:
        frequency_dict[character] += 1
    return sorted(frequency_dict.items(), key=lambda x: x[1], reverse=True)


def generate_codes(node, is_left=True, binary_string=''):
    if type(node) is str:
        return {node: binary_string}
    left, right = node.get_children()
    code_dict = {}
    code_dict.update(generate_codes(left, True, binary_string + '0'))
    code_dict.update(generate_codes(right, False, binary_string + '1'))
    return code_dict


def build_tree(text):
    nodes = calculate_frequency(text, alphabet)

    while len(nodes) > 1:
        (char1, freq1) = nodes[-1]
        (char2, freq2) = nodes[-2]
        nodes = nodes[:-2]
        new_node = BinaryTree(char1, char2)
        nodes.append((new_node, freq1 + freq2))
        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

    huffman_codes = generate_codes(nodes[0][0])

    return huffman_codes


def decode_text(encoded_text, codes):
    decoded_output = ''
    reverse_codes = {v: k for k, v in codes.items()}
    start_index = 0
    for i in range(0, len(encoded_text) + 1):
        if encoded_text[start_index:i] in reverse_codes:
            decoded_output += reverse_codes[encoded_text[start_index:i]]
            start_index = i
    return decoded_output


def encode_text(text, codes):
    encoded_output = ""
    for character in text:
        encoded_output += codes[character]
    return encoded_output


def save_to_files(code_filename, codes, binary_filename, encoded_text):
    with open(code_filename, 'w') as code_file:
        for character, binary in codes.items():
            code_file.write(character + ";" + str(binary) + ";")

    bits = bitarray.bitarray(encoded_text)

    with open(binary_filename, 'wb') as binary_file:
        bits.tofile(binary_file)


def load_from_files(code_filename, binary_filename, text_length):
    with open(code_filename) as code_file:
        code_data = code_file.read()
    split_codes = code_data.split(";")
    codes = {split_codes[i]: split_codes[i + 1] for i in range(0, len(split_codes) - 1, 2)}

    bits = bitarray.bitarray()
    with open(binary_filename, 'rb') as binary_file:
        bits.fromfile(binary_file)
    loaded_text = bits.to01()[:text_length]

    return loaded_text, codes


def compute_compression_ratio(text, codebook):
    uncompressed_size = len(text) * 8
    compressed_size = sum(len(codebook[char]) for char in text)
    return uncompressed_size / compressed_size


def compute_average_length(codes, text_length, frequency_dict):
    average_length = np.sum([(freq / text_length) * len(codes[char]) for char, freq in frequency_dict])
    return average_length




In [11]:
input_text = open("norm_wiki_sample.txt").read()
codebook = build_tree(input_text)
encoded_text = encode_text(input_text, codebook)
decoded_text = decode_text(encoded_text, codebook)

print("Is the original text equal to the decoded text?", input_text == decoded_text)

save_to_files("codebook.txt", codebook, "encoded.bin", encoded_text)

text_length = len(encoded_text)
loaded_encoded_text, loaded_codebook = load_from_files("codebook.txt", "encoded.bin", text_length)
decoded_text_from_file = decode_text(loaded_encoded_text, loaded_codebook)

print("Is the original text equal to the decoded text (from file)?", input_text == decoded_text_from_file)
print("Compression ratio:", compute_compression_ratio(input_text, codebook))

frequency_dict = calculate_frequency(input_text, alphabet)
average_code_length = compute_average_length(codebook, len(input_text), frequency_dict)
print("Average length of code words:", average_code_length)

text_entropy = np.sum([(freq / len(input_text)) * np.log2(freq / len(input_text)) for _, freq in frequency_dict]) * -1
print("Entropy:", text_entropy)
print("Coding efficiency:", text_entropy / average_code_length)

Is the original text equal to the decoded text? True
Is the original text equal to the decoded text (from file)? True
Compression ratio: 1.8565725743118144
Average length of code words: 4.3090155002237935
Entropy: 4.2803962467015655
Coding efficiency: 0.9933582848516693
